In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from tabulate import tabulate
from itertools import product

# Modelos de Machine Learning
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import xgboost as xgb

# Preprocesamiento
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Model Selection y Validación
from sklearn.model_selection import (
    train_test_split, cross_val_score, learning_curve, validation_curve
)

# Métricas
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, roc_auc_score,
    roc_curve, precision_recall_curve, precision_score, recall_score, f1_score,confusion_matrix, ConfusionMatrixDisplay
)


In [2]:
# Cargar el archivo CSV limpio
data_limpia = pd.read_csv('../data/data_limpia.csv')

# Ver las primeras filas del DataFrame para confirmar que se ha cargado correctamente
data_limpia.head()

,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,0,0,0,0,1,1,0,0,1,1,...,2,0,2,1,2892,0,0,2,18,0
1,11111,0,0,0,1,1,0,0,0,1,...,2,1,3,0,1376,7187,0,2,53,0
2,22222,0,0,0,1,1,3,0,1,1,...,1,0,3,0,1376,0,1,4,21,3
3,33333,0,0,0,1,1,2,0,0,1,...,1,1,2,1,2015,7256,1,4,48,5
4,53605,0,0,0,1,1,2,0,0,1,...,1,1,1,0,2015,11293,0,2,56,0


In [3]:
# Definir las características (features) y el objetivo (target)
X = data_limpia.drop(columns=['Mas_1_coche', 'Tiempo'])  # Eliminamos también la columna 'Tiempo'
y = data_limpia['Mas_1_coche']  # Objetivo (si el cliente comprará un segundo coche)

In [4]:
# Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from itertools import product
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score

# Definir los parámetros para la búsqueda de Grid (con parámetros ajustados)
param_grid = {
    'n_estimators': [200],  # Reducir los estimadores a 100 o 200
    'learning_rate': [0.05, 0.1],  # Reducir la tasa de aprendizaje
    'max_depth': [16, 18],  # Profundidad máxima de los árboles
}

# Almacenar los resultados
results = []

# Modelos a probar
models = {
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
}

# Iterar sobre cada modelo y cada combinación de hiperparámetros
for model_name, model in models.items():
    for params in product(*param_grid.values()):
        n_estimators, learning_rate, max_depth = params
        
        # Asegurarse de que max_depth sea un valor entero para XGBoost
        max_depth = int(max_depth)
        
        # Configurar el modelo con los parámetros actuales
        if model_name == 'AdaBoost':
            model.set_params(n_estimators=n_estimators, learning_rate=learning_rate)
        elif model_name == 'GradientBoosting':
            model.set_params(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth)
        elif model_name == 'XGBoost':
            model.set_params(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth)
        
        # Entrenar el modelo
        model.fit(X_train, y_train)
        
        # Predecir en el conjunto de prueba
        y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades para la clase positiva
        y_pred = (y_pred_proba > 0.4).astype(int)  # Ajustar umbral de decisión
        
        # Calcular métricas
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')  # Enfocarse en recall
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        
        # Validación cruzada
        cv_scores = cross_val_score(model, X_train, y_train, cv=3, scoring='recall')  # Optimizar recall
        mean_cv_score = np.mean(cv_scores)
        
        # Guardar los resultados
        results.append({
            'model': model_name,
            'n_estimators': n_estimators,
            'learning_rate': learning_rate,
            'max_depth': max_depth if model_name != 'AdaBoost' else None,
            'accuracy': accuracy,
            'f1_score': f1,
            'recall': recall,
            'roc_auc': roc_auc,
            'cv_recall': mean_cv_score  # Guardar recall en validación cruzada
        })

# Convertir a DataFrame y ordenar por recall
results_df = pd.DataFrame(results).sort_values(by=['recall', 'f1_score'], ascending=False)

# Mostrar los mejores modelos optimizados para recall
display(results_df.head(5))

# Graficar las métricas: Curva ROC y Recall vs Umbral para el mejor modelo
best_model = results_df.iloc[0]
best_model_params = best_model.drop(['accuracy', 'f1_score', 'recall', 'roc_auc', 'cv_recall']).to_dict()

# Crear el mejor modelo con los parámetros óptimos
if best_model['model'] == 'AdaBoost':
    best_model_obj = AdaBoostClassifier(
        n_estimators=best_model_params['n_estimators'],
        learning_rate=best_model_params['learning_rate'],
        random_state=42
    )
elif best_model['model'] == 'GradientBoosting':
    best_model_obj = GradientBoostingClassifier(
        n_estimators=best_model_params['n_estimators'],
        learning_rate=best_model_params['learning_rate'],
        max_depth=best_model_params['max_depth'],
        random_state=42
    )
else:  # XGBoost
    best_model_obj = xgb.XGBClassifier(
        n_estimators=best_model_params['n_estimators'],
        learning_rate=best_model_params['learning_rate'],
        max_depth=int(best_model_params['max_depth']),  # Asegurarse de que max_depth sea int
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )

# Entrenar el mejor modelo
best_model_obj.fit(X_train, y_train)

# Calcular las probabilidades de la clase positiva
y_pred_proba = best_model_obj.predict_proba(X_test)[:, 1]

# Calcular la curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Graficar la Curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc_score(y_test, y_pred_proba):.2f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.title(f'Curva ROC - {best_model["model"]}')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# Graficar Recall vs Umbral
plt.figure(figsize=(8, 6))
plt.plot(thresholds, tpr, label='Recall')
plt.title('Recall vs Umbral de Decisión')
plt.xlabel('Umbral de Decisión')
plt.ylabel('Recall')
plt.grid(True)
plt.show()

# Matriz de Confusión
y_pred = best_model_obj.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

# Mostrar la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Clase 0', 'Clase 1'])
disp.plot(cmap=plt.cm.Blues)
plt.title(f'Matriz de Confusión - {best_model["model"]}')
plt.show()
